<a href="https://colab.research.google.com/github/netra-poonia/simulation1/blob/main/Branch_Manager_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Base Code

In [ ]:
import csv
import math

In [ ]:
class BMIncentiveCalculator:

    def __init__(self):
        self.secured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 2): 1400, (3, 4): 1800, (5, 6): 2200, (7, float('inf')): 2400},
            (1, 6): {(0, 0): 1200, (1, 2): 1600, (3, 4): 2000, (5, 6): 2500, (7, float('inf')): 2800},
            (7, 9): {(0, 0): 1400, (1, 2): 1800, (3, 4): 2500, (5, 6): 3000, (7, float('inf')): 4000},
            (10, float('inf')): {(0, 0): 1800, (1, 2): 2200, (3, 4): 3000, (5, 6): 3500, (7, float('inf')): 5000}
        }

        self.unsecured_fresh_disbursals = {
            (0, 0): {(0, 0): 0, (1, 3): 250, (4, 7): 350, (8, 9): 450, (10, 13): 550, (14, float('inf')): 700},
            (1, 14): {(0, 0): 200, (1, 3): 300, (4, 7): 400, (8, 9): 500, (10, 13): 600, (14, float('inf')): 750},
            (15, 19): {(0, 0): 250, (1, 3): 350, (4, 7): 450, (8, 9): 550, (10, 13): 650, (14, float('inf')): 850},
            (20, 23): {(0, 0): 300, (1, 3): 400, (4, 7): 500, (8, 9): 600, (10, 13): 700, (14, float('inf')): 1000},
            (24, float('inf')): {(0, 0): 500, (1, 3): 600, (4, 7): 700, (8, 9): 1000, (10, 13): 1200, (14, float('inf')): 1500}
        }

        self.secured_business_slabs = [
            (0, 60, 0),
            (60, 80, 450),
            (80, 100, 500),
            (100, 150, 600),
            (150, 1000, 750)
        ]

        self.secured_direct_business_slabs = [
            (0, 40, 50),
            (40, 60, 100),
            (60, 80, 150),
            (80, 1000, 200)
        ]

        self.unsecured_business_slabs = [
            (0, 60, 0),
            (60, 80, 300),
            (80, 100, 400),
            (100, 150, 500),
            (150, 1000, 600)
        ]

    def _get_bonus_from_matrix(self, matrix, logins, disbursals):
        # print(f"\n--- Effort Bonus Calculation (Logins: {logins}, Disbursals: {disbursals}) ---")
        for login_range, disbursal_bonuses in matrix.items():
            login_min, login_max = login_range
            # print(f"Checking login range: ({login_min}, {login_max})")
            if login_min <= logins <= login_max:
                # print(f"  Login {logins} falls in range ({login_min}, {login_max}).")
                for disbursal_range, bonus_amount in disbursal_bonuses.items():
                    disbursal_min, disbursal_max = disbursal_range
                    # print(f"    Checking disbursal range: ({disbursal_min}, {disbursal_max})")
                    if disbursal_min <= disbursals <= disbursal_max:
                        # print(f"      Disbursal {disbursals} falls in range ({disbursal_min}, {disbursal_max}).")
                        # print(f"      Matched bonus amount: {bonus_amount}")
                        return bonus_amount
        print("  No matching bonus found for given logins and disbursals.")
        return 0

    def _calculate_volume_bonus_tax_slab(self, total_disbursal_amount, target_amount, slabs):
        print(f"\n--- Volume Bonus Calculation (Amount: {total_disbursal_amount}, Target: {target_amount}) ---")
        if target_amount <= 0:
            print("Target amount is zero or less, volume bonus is 0.")
            return 0

        achievement_percentage = (total_disbursal_amount / target_amount) * 100
        print(f"Achievement Percentage: {achievement_percentage:.2f}%")

        effective_rate = 0
        first_non_zero_slab_lower_bound_amount = 0
        found_first_non_zero = False

        # Sort slabs by lower bound of percentage for correct sequential processing
        sorted_slabs = sorted(slabs, key=lambda x: x[0])

        for lower_perc, upper_perc, rate in sorted_slabs:
            # Calculate the actual amount thresholds for this slab based on the target
            lower_bound_amount = (lower_perc / 100.0) * target_amount
            upper_bound_amount = (upper_perc / 100.0) * target_amount

            print(f"  Slab: ({lower_perc:.0f}%, {upper_perc:.0f}%) | Rate: {rate} | Lower Amt: {lower_bound_amount:.2f} | Upper Amt: {upper_bound_amount:.2f}")

            # Determine the effective rate based on the highest slab achieved
            if lower_perc <= achievement_percentage <= upper_perc:
                effective_rate = rate
                print(f"    Achievement {achievement_percentage:.2f}% falls in this slab. Effective rate set to {effective_rate}.")

            # Find the lower bound of the first slab with a non-zero rate
            if rate > 0 and not found_first_non_zero:
                first_non_zero_slab_lower_bound_amount = lower_bound_amount
                found_first_non_zero = True
                print(f"    First non-zero rate slab starts at {first_non_zero_slab_lower_bound_amount:.2f}.")

        volume_bonus = 0
        if effective_rate > 0:
            # Calculate bonus on the amount above the lower bound of the first non-zero rate slab
            # Ensure we don't calculate bonus on negative amounts if disbursed amount is less than threshold
            amount_for_bonus = max(0, total_disbursal_amount - first_non_zero_slab_lower_bound_amount)
            volume_bonus = (amount_for_bonus / 100000.0) * effective_rate
            print(f"  Amount eligible for bonus (above first non-zero slab threshold): {amount_for_bonus:.2f}")
            print(f"  Calculated Volume Bonus: ({amount_for_bonus:.2f} / 100000.0) * {effective_rate} = {volume_bonus:.2f}")
        else:
            print("  No effective rate or achievement too low for bonus.")

        return volume_bonus

    def _calculate_volume_bonus_simple_rate(self, total_disbursal_amount, achievement_percentage, slabs):
        # print(f"\n--- Simple Rate Volume Bonus Calculation (Amount: {total_disbursal_amount}, Percentage: {achievement_percentage:.2f}%) ---")
        for lower_perc, upper_perc, rate in slabs:
            # print(f"  Checking slab: ({lower_perc}%, {upper_perc}%) | Rate: {rate}")
            if lower_perc <= achievement_percentage <= upper_perc:
                bonus = (total_disbursal_amount / 100000.0) * rate
                # print(f"  Achievement {achievement_percentage:.2f}% falls in range ({lower_perc}%, {upper_perc}%).")
                # print(f"  Calculated Bonus: ({total_disbursal_amount:.2f} / 100000.0) * {rate} = {bonus:.2f}")
                return bonus
        print("  No matching bonus rate found for the achievement percentage.")
        return 0

    def calculate_bonus(self, secured_logins, unsecured_logins, secured_disbursals,
                        unsecured_disbursals, total_secured_disbursal_amount,
                        secured_direct_percentage, secured_target, unsecured_target,
                        total_unsecured_disbursal_amount=0):
        # --- Effort Bonus Calculation ---
        secured_effort_bonus = self._get_bonus_from_matrix(
            self.secured_fresh_disbursals, secured_logins, secured_disbursals
        )

        unsecured_effort_bonus = self._get_bonus_from_matrix(
            self.unsecured_fresh_disbursals, unsecured_logins, unsecured_disbursals
        )

        total_effort_bonus = secured_effort_bonus + unsecured_effort_bonus

        # --- Volume Bonus Calculation ---

        # Secured Business Volume Bonus (using stepped-up bonus logic)
        secured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_secured_disbursal_amount, secured_target, self.secured_business_slabs
        )

        # Secured Direct Business Volume Bonus (using simple rate logic)
        secured_direct_volume_bonus = self._calculate_volume_bonus_simple_rate(
            total_secured_disbursal_amount, secured_direct_percentage, self.secured_direct_business_slabs
        )

        # Unsecured Business Volume Bonus (using stepped-up bonus logic, if applicable)
        unsecured_volume_bonus = self._calculate_volume_bonus_tax_slab(
            total_unsecured_disbursal_amount, unsecured_target, self.unsecured_business_slabs
        )

        total_volume_bonus = secured_volume_bonus + secured_direct_volume_bonus + unsecured_volume_bonus

        total_bonus = total_effort_bonus + total_volume_bonus

        return {
            "secured_effort_bonus": secured_effort_bonus,
            "unsecured_effort_bonus": unsecured_effort_bonus,
            "total_effort_bonus": total_effort_bonus,
            "secured_volume_bonus": secured_volume_bonus,
            "secured_direct_volume_bonus": secured_direct_volume_bonus,
            "unsecured_volume_bonus": unsecured_volume_bonus,
            "total_volume_bonus": total_volume_bonus,
            "overall_total_bonus": total_bonus
        }

In [ ]:
def process_bonus_csv(input_csv_path, output_csv_path):
    calculator = BMIncentiveCalculator()

    input_headers_mapping = {
        'secured_logins': 'secured_logins',
        'unsecured_logins': 'unsecured_logins',
        'secured_disbursals': 'secured_disbursals',
        'unsecured_disbursals': 'unsecured_disbursals',
        'total_secured_disbursal_amount': 'total_secured_disbursal_amount',
        'secured_direct_percentage': 'secured_direct_percentage',
        'secured_target': 'secured_target',
        'unsecured_target': 'unsecured_target',
        'total_unsecured_disbursal_amount': 'total_unsecured_disbursal_amount'
    }

    output_bonus_headers = [
        'secured_effort_bonus',
        'unsecured_effort_bonus',
        'total_effort_bonus',
        'secured_volume_bonus',
        'secured_direct_volume_bonus',
        'unsecured_volume_bonus',
        'total_volume_bonus',
        'overall_total_bonus'
    ]

    all_data = []
    original_headers = []

    try:
        with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)
            original_headers = reader.fieldnames # Get original headers

            if not original_headers:
                print(f"Error: Input CSV '{input_csv_path}' is empty or has no headers.")
                return

            # Check if all required headers are present
            missing_headers = [header for param, header in input_headers_mapping.items() if header not in original_headers]
            if missing_headers:
                print(f"Error: Missing required headers in input CSV: {', '.join(missing_headers)}")
                print(f"Expected headers (from mapping): {list(input_headers_mapping.values())}")
                print(f"Available headers in CSV: {original_headers}")
                return

            for row_num, row in enumerate(reader, 2): # Start row_num from 2 for data rows
                try:
                    secured_logins = int(row.get(input_headers_mapping['secured_logins'], 0))
                    unsecured_logins = int(row.get(input_headers_mapping['unsecured_logins'], 0))
                    secured_disbursals = int(row.get(input_headers_mapping['secured_disbursals'], 0))
                    unsecured_disbursals = int(row.get(input_headers_mapping['unsecured_disbursals'], 0))
                    total_secured_disbursal_amount = float(row.get(input_headers_mapping['total_secured_disbursal_amount'], 0.0))
                    secured_direct_percentage = float(row.get(input_headers_mapping['secured_direct_percentage'], 0.0))
                    secured_target = float(row.get(input_headers_mapping['secured_target'], 0.0))
                    unsecured_target = float(row.get(input_headers_mapping['unsecured_target'], 0.0))
                    total_unsecured_disbursal_amount = float(row.get(input_headers_mapping['total_unsecured_disbursal_amount'], 0.0))

                    bonus_results = calculator.calculate_bonus(
                        secured_logins=secured_logins,
                        unsecured_logins=unsecured_logins,
                        secured_disbursals=secured_disbursals,
                        unsecured_disbursals=unsecured_disbursals,
                        total_secured_disbursal_amount=total_secured_disbursal_amount,
                        secured_direct_percentage=secured_direct_percentage,
                        secured_target=secured_target,
                        unsecured_target=unsecured_target,
                        total_unsecured_disbursal_amount=total_unsecured_disbursal_amount
                    )

                    for key, value in bonus_results.items():
                        row[key] = f"{value:.2f}"

                    all_data.append(row)

                except ValueError as e:
                    print(f"Error converting data in row {row_num}: {e}. Skipping row: {row}")
                except KeyError as e:
                    print(f"Error: Missing key in row {row_num}: {e}. Check CSV headers. Skipping row: {row}")
                except Exception as e:
                    print(f"An unexpected error occurred processing row {row_num}: {e}. Skipping row: {row}")

        output_headers = original_headers + output_bonus_headers

        with open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=output_headers)
            writer.writeheader()
            writer.writerows(all_data)

        print(f"\nSuccessfully processed data from '{input_csv_path}' and saved results to '{output_csv_path}'")

    except FileNotFoundError:
        print(f"Error: Input CSV file not found at '{input_csv_path}'")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

## One on one output

In [ ]:
# --- Example Usage ---
calculator = BMIncentiveCalculator()

secured_logins = 23
unsecured_logins = 5
secured_disbursals = 9
unsecured_disbursals = 3
total_secured_disbursal_amount = 4995395
secured_direct_percentage = 100
secured_target =  3325000
unsecured_target =  175000
total_unsecured_disbursal_amount_example = 700000

bonus_results = calculator.calculate_bonus(
    secured_logins=secured_logins,
    unsecured_logins=unsecured_logins,
    secured_disbursals=secured_disbursals,
    unsecured_disbursals=unsecured_disbursals,
    total_secured_disbursal_amount=total_secured_disbursal_amount,
    secured_direct_percentage=secured_direct_percentage,
    secured_target=secured_target,
    unsecured_target=unsecured_target,
    total_unsecured_disbursal_amount=total_unsecured_disbursal_amount_example
)

# Print the results
print("\n--- Final Bonus Calculation Results ---")
print(f"Secured Effort Bonus: {bonus_results['secured_effort_bonus']:.2f}")
print(f"Unsecured Effort Bonus: {bonus_results['unsecured_effort_bonus']:.2f}")
print(f"Total Effort Bonus: {bonus_results['total_effort_bonus']:.2f}")
print("-" * 30)
print(f"Secured Volume Bonus (General): {bonus_results['secured_volume_bonus']:.2f}")
print(f"Secured Direct Volume Bonus: {bonus_results['secured_direct_volume_bonus']:.2f}")
print(f"Unsecured Volume Bonus: {bonus_results['unsecured_volume_bonus']:.2f}")
print(f"Total Volume Bonus: {bonus_results['total_volume_bonus']:.2f}")
print("-" * 30)
print(f"Overall Total Bonus: {bonus_results['overall_total_bonus']:.2f}")


--- Volume Bonus Calculation (Amount: 4995395, Target: 3325000) ---
Achievement Percentage: 150.24%
  Slab: (0%, 60%) | Rate: 0 | Lower Amt: 0.00 | Upper Amt: 1995000.00
  Slab: (60%, 80%) | Rate: 450 | Lower Amt: 1995000.00 | Upper Amt: 2660000.00
    First non-zero rate slab starts at 1995000.00.
  Slab: (80%, 100%) | Rate: 500 | Lower Amt: 2660000.00 | Upper Amt: 3325000.00
  Slab: (100%, 150%) | Rate: 600 | Lower Amt: 3325000.00 | Upper Amt: 4987500.00
  Slab: (150%, 1000%) | Rate: 750 | Lower Amt: 4987500.00 | Upper Amt: 33250000.00
    Achievement 150.24% falls in this slab. Effective rate set to 750.
  Amount eligible for bonus (above first non-zero slab threshold): 3000395.00
  Calculated Volume Bonus: (3000395.00 / 100000.0) * 750 = 22502.96

--- Volume Bonus Calculation (Amount: 700000, Target: 175000) ---
Achievement Percentage: 400.00%
  Slab: (0%, 60%) | Rate: 0 | Lower Amt: 0.00 | Upper Amt: 105000.00
  Slab: (60%, 80%) | Rate: 300 | Lower Amt: 105000.00 | Upper Amt: 140

## File output

In [ ]:
if __name__ == "__main__":
    dummy_data = [
        ['Branch_ID', 'Manager_Name', 'secured_logins', 'unsecured_logins', 'secured_disbursals',
         'unsecured_disbursals', 'total_secured_disbursal_amount', 'secured_direct_percentage',
         'secured_target', 'unsecured_target', 'total_unsecured_disbursal_amount'],
        ['B001', 'Alice', '14', '1', '5', '0', '3906813', '100', '4275000', '225000', '0'],
        ['B002', 'Bob', '23', '5', '9', '3', '4995395', '100', '3325000', '175000', '700000'],
        ['B003', 'Charlie', '20', '0', '1', '0', '2000000', '100', '3325000', '175000', '0']
    ]

    dummy_input_csv_path = 'input_bonus_data.csv'
    with open(dummy_input_csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(dummy_data)

    print(f"Created a dummy input CSV: {dummy_input_csv_path}")

    # Set your desired output CSV file path
    output_csv_file = 'output_bonus_results.csv'

    # Run the processing function
    process_bonus_csv(dummy_input_csv_path, output_csv_file)

Created a dummy input CSV: input_bonus_data.csv
  No effective rate or achievement too low for bonus.
  No effective rate or achievement too low for bonus.

Successfully processed data from 'input_bonus_data.csv' and saved results to 'output_bonus_results.csv'
